### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,41):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  800 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n10\n\n\nStory\n10\n\n\nAnimat...
1,\n\n\n\nOverall\n9\n\n\nStory\n8\n\n\nAnimatio...
2,\n\n\n\nOverall\n7\n\n\nStory\n8\n\n\nAnimatio...
3,\n\n\n\nOverall\n7\n\n\nStory\n8\n\n\nAnimatio...
4,\n\n\n\nOverall\n3\n\n\nStory\n2\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


 Overall 10 Story 10 Animation 9 Sound 9 Character 10 Enjoyment 10 First of all, I have seen the original FMA and although it was very popular and original, the pacing and conclusion did not sit too well with me. Brotherhood is meant to be a remake of the original, this time sticking to the manga all the way through, but there were people who thought it would spoil the franchise. That myth should be dispelled, as there's only one word to describe this series - EPIC. I admit that as I've seen the original and read the manga, the pacing of Brotherhood seems to start off being VERY fast (I finally got used to the pacing after watching the first fifteen eps or so). Events that took up half a volume of the manga and had spread though a few episodes of the original anime were now shown in just a single episode. However, after trying to look at it from the perspective of someone who's new to FMA (not comparing it to the manga nor the original), I believe that the pacing works and it manages t

In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'first', 'of', 'all', 'have', 'seen', 'the', 'original', 'fma', 'and', 'although', 'it', 'was', 'very', 'popular', 'and', 'original', 'the', 'pacing', 'and', 'conclusion', 'did', 'not', 'sit', 'too', 'well', 'with', 'me', 'brotherhood', 'is', 'meant', 'to', 'be', 'remake', 'of', 'the', 'original', 'this', 'time', 'sticking', 'to', 'the', 'manga', 'all', 'the', 'way', 'through', 'but', 'there', 'were', 'people', 'who', 'thought', 'it', 'would', 'spoil', 'the', 'franchise', 'that', 'myth', 'should', 'be', 'dispelled', 'as', 'there', 'only', 'one', 'word', 'to', 'describe', 'this', 'series', 'epic', 'admit', 'that', 'as', 've', 'seen', 'the', 'original', 'and', 'read', 'the', 'manga', 'the', 'pacing', 'of', 'brotherhood', 'seems', 'to', 'start', 'off', 'being', 'very', 'fast', 'finally', 'got', 'used', 'to', 'the', 'pacing', 'after', 'watching', 'the', 'first', 'fifteen', 'eps', 'or', 'so', 'events', 'that', 'took', 'up'

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'spoiler', 'warning', 'aside', 'from', 'the', 'usual', 'everyday', 'anime', 'art', 'and', 'sound', 'standard', 'of', 'which', 'most', 'animes', 'pass', 'anyways', 'there', 'are', 'essentially', 'three', 'main', 'factors', 'that', 'see', 'as', 'important', 'in', 'anime', 'complexity', 'detail', 'which', 'adds', 'depth', 'characters', 'plot', 'setting', 'progress', 'which', 'adds', 'depth', 'character', 'development', 'plot', 'development', 'shifts', 'of', 'setting', 'novelty', 'which', 'again', 'adds', 'depth', 'characters', 'personality', 'setting', 'plot', 'complexity', 'detail', 'it', 'understandable', 'that', 'because', 'fma', 'brotherhood', 'paid', 'attention', 'to', 'its', 'simple', 'minded', 'viewers', 'readers', 'the', 'resulting', 'complexity', 'of', 'the', 'entirety', 'of', 'the', 'anime', 'would', 'turn', 'out', 'bland', 'the', 'characters', 'were', 'too', 'simple', 'particular', 'the', 'main', 'characters',

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'first', 'see', 'original', 'fma', 'popular', 'original', 'pacing', 'conclusion', 'sit', 'well', 'brotherhood', 'mean', 'remake', 'original', 'time', 'stick', 'manga', 'way', 'people', 'think', 'spoil', 'franchise', 'myth', 'dispel', 'word', 'describe', 'series', 'epic', 'admit', 'see', 'original', 'read', 'manga', 'pace', 'brotherhood', 'seem', 'start', 'fast', 'finally', 'get', 'use', 'pace', 'watch', 'first', 'ep', 'event', 'take', 'half', 'volume', 'manga', 'spread', 'episode', 'original', 'anime', 'show', 'single', 'episode', 'however', 'try', 'look', 'perspective', 'new', 'fma', 'compare', 'manga', 'original', 'believe', 'pace', 'work', 'manage', 'tell', 'intriguing', 'story', 'effectively', 'little', 'confusion', 'plot', 'full', 'clever', 'idea', 'unpredictable', 'twist', 'link', 'various', 'part', 'story', 'together', 'final', 'episode', 'loose_end', 'neatly', 'tie', 'leave', 'hugely', 'satisfy', 'epilogue', '

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 1), (9, 2), (10, 5), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 6), (25, 1), (26, 1), (27, 1), (28, 13), (29, 2), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 3), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 8), (66, 1), (67, 2), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 2), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 4), (91, 1), (92, 1), (93, 1), (94, 9), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 4), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [21]:
# Using LDA to develop a topic model with 3 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 3 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.001*"disjoint" + 0.001*"stylistic" + 0.001*"exquisitely" + 0.000*"bane" + 0.000*"antic" + 0.000*"reconciliation" + 0.000*"commentate" + 0.000*"endpoint" + 0.000*"pretentiously" + 0.000*"sugarcoat"'), (1, '0.011*"alchemy" + 0.010*"edward" + 0.009*"get" + 0.009*"make" + 0.009*"brother" + 0.008*"human" + 0.007*"show" + 0.006*"fight" + 0.006*"body" + 0.006*"alphonse"'), (2, '0.000*"grader" + 0.000*"prayer" + 0.000*"righ" + 0.000*"yaaay" + 0.000*"implore" + 0.000*"oozed" + 0.000*"cave" + 0.000*"conclusive" + 0.000*"alche" + 0.000*"appreciation"'), (3, '0.032*"character" + 0.027*"anime" + 0.024*"story" + 0.016*"watch" + 0.014*"show" + 0.013*"good" + 0.013*"well" + 0.012*"brotherhood" + 0.011*"series" + 0.011*"sound"'), (4, '0.000*"grader" + 0.000*"prayer" + 0.000*"righ" + 0.000*"yaaay" + 0.000*"implore" + 0.000*"oozed" + 0.000*"cave" + 0.000*"conclusive" + 0.000*"alche" + 0.000*"appreciation"'), (5, '0.025*"el" + 0.018*"s" + 0.008*"fuck" + 0.007*"ve" + 0.007*"hagaren" + 0.007*"aswell

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.331220 -0.152509       1        1  80.130730
1     -0.272789  0.171455       2        1  18.849718
7      0.154398 -0.031642       3        1   0.445994
5      0.108530  0.002658       4        1   0.339214
6      0.105219  0.002442       5        1   0.203842
0      0.079128  0.002524       6        1   0.020376
4      0.078367  0.002536       7        1   0.005207
2      0.078367  0.002536       8        1   0.004918, topic_info=          Term         Freq        Total Category  logprob  loglift
10       anime  3653.000000  3653.000000  Default  30.0000  30.0000
4      alchemy   488.000000   488.000000  Default  29.0000  29.0000
348     edward   517.000000   517.000000  Default  28.0000  28.0000
264      watch  2087.000000  2087.000000  Default  27.0000  27.0000
23     brother   465.000000   465.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
25        call     0.000856   151.565477   Topic8  -9.1677  -2.1642
26        cast     0.000856   167.159525   Topic8  -9.1677  -2.2621
27       chang     0.000856     5.360289   Topic8  -9.1677   1.1778
28   character     0.000856  4417.467864   Topic8  -9.1677  -5.5365
29      clever     0.000856    16.771152   Topic8  -9.1677   0.0372

[421 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.900344  absolutely
0         2  0.097537  absolutely
1         1  0.937261      action
1         2  0.062484      action
2         1  0.845113      active
...     ...       ...         ...
269       1  0.572360       world
269       2  0.429270       world
2405      1  1.000677       worth
4577      4  0.898679         wtf
1322      2  0.828101        zany

[345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 8, 6, 7, 1, 5, 3])

In [23]:
# Save the data frame as a CSV file
# job_datas.to_csv(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Programming Assignment 2\Chan_Matthew_INST447_PA2.csv')

In [24]:
# Week 9 Topic Modelling
# ner_corpus = ' '.join(ner_df['Name'])
# corp = open(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Week 9\corpus.txt',"w")
# corp.writelines(ner_corpus)
# corp.close()